## Task 1: Entity Resolution (Record Linkage) 

#### Required libraries

The Record Linkage ToolKit ([RLTK](https://github.com/usc-isi-i2/rltk)) is a general-purpose open-source record linkage platform that allows users to build powerful Python programs that link records referring to the same underlying entity. You can also find additional examples and use-cases in [RLTK's documentation](https://rltk.readthedocs.io/en/master/).

In [1]:
import rltk
import csv
import pandas as pd
import csv
from rdflib import Graph, URIRef, Literal, XSD, Namespace, RDF
import numpy as np
import rdflib
import io
import os
from datetime import datetime
import pandas as pd
from IPython.display import display
from difflib import SequenceMatcher
import rltk
import re
import warnings
warnings.filterwarnings("ignore")

### Construct RLTK Datasets

#### Read csv file and set a tokenizer

In [2]:
sizeDict = {'10001+' : '10000+ Employees', 
            '5001-10000' : '5001 to 10000 Employees',
            '1001-5000' : '5001 to 10000 Employees', 
            '501-1000': '501 to 1000 Employees', 
            '251-500' : '201 to 500 Employees',
            '101-250': '51 to 200 Employees', 
            '51-100' : '51 to 200 Employees', 
            '11-50' : '1 to 50 Employees', 
            '1-10' : '1 to 50 Employees'
}

In [3]:
dir_ = './data/'
crunchbase = dir_ + 'crunchbase_companyInfo2.csv'
glassdoor = dir_ + 'glassdoor_companyInfo2.csv'

# Read csv file to check the shape of data
crunch = pd.read_csv(crunchbase)
glass = pd.read_csv(glassdoor)

# You can use this tokenizer in case you need to manipulate some data
tokenizer = rltk.tokenizer.crf_tokenizer.crf_tokenizer.CrfTokenizer()

#### Select attributes that are the most useful for the record linkage task. Then construct RLTK datasets with the selected fields.

In [4]:
class CrunchRecord(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = ''

    @rltk.cached_property
    def id(self):
        return self.raw_object['Unnamed: 0']
    
    @rltk.cached_property
    def name_string(self):
        return self.raw_object['name']

    @rltk.cached_property
    def name_tokens(self):
        return set(tokenizer.tokenize(self.name_string))
    
    @rltk.cached_property
    def h_city_string(self):
        return self.raw_object['headquaters_location_city']
    
    @rltk.cached_property
    def h_state_string(self):
        return self.raw_object['headquaters_location_state']
    
    @rltk.cached_property
    def link_string(self):
        return self.raw_object['website_link']

    @rltk.cached_property
    def size_string(self):\
        return self.raw_object['num_employees_enum']
    
    @rltk.cached_property
    def industry_string(self):
        return self.raw_object['industries']

# Read dataset as rltk type
ds1 = rltk.Dataset(rltk.CSVReader(crunchbase), record_class=CrunchRecord)
print(ds1.generate_dataframe().head(2))

  id     name_string        name_tokens h_city_string h_state_string  \
0  0  1QVIA Holdings  {1QVIA, Holdings}       Danbury    Connecticut   
1  1              3M               {3M}    Saint Paul      Minnesota   

              link_string            size_string  \
0  https://www.iqvia.com/  501 to 1000 Employees   
1      https://www.3m.com       10000+ Employees   

                                     industry_string  
0       ['Analytics', 'Health Care', 'Life Science']  
1  ['Automotive', 'Electronics', 'Enterprise Soft...  


In [5]:
class GlassRecord(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = ''

    @rltk.cached_property
    def id(self):
        return self.raw_object['Unnamed: 0']
    
    @rltk.cached_property
    def name_string(self):
        return self.raw_object['company']
    
    @rltk.cached_property
    def name_tokens(self):
        return set(tokenizer.tokenize(self.name_string))
    
    @rltk.cached_property
    def headquaters_string(self):
        return self.raw_object['headquarters']
    
    @rltk.cached_property
    def h_city_string(self):
        return self.headquaters_string.split(',')[0]
    
    @rltk.cached_property
    def h_state_string(self):
        if len(self.headquaters_string.split(',')) == 2:
            return self.headquaters_string.split(',')[1]
        else:
            return ''
    
    @rltk.cached_property
    def link_string(self):
        return self.raw_object['website']

    @rltk.cached_property
    def size_string(self):
        return self.raw_object['size']
    
    @rltk.cached_property
    def industry_string(self):
        return self.raw_object['industry'].split(' & ')
    
ds2 = rltk.Dataset(rltk.CSVReader(glassdoor),record_class=GlassRecord)
print(ds2.generate_dataframe().head(2))
print()

  id name_string name_tokens headquaters_string h_city_string h_state_string  \
0  0     Walmart   {Walmart}    Bentonville, AR   Bentonville             AR   
1  1      Amazon    {Amazon}        Seattle, WA       Seattle             WA   

           link_string       size_string                     industry_string  
0  careers.walmart.com  10000+ Employees  [General Merchandise, Superstores]  
1      www.amazon.jobs  10000+ Employees            [Internet, Web Services]  



### Entity Linking

#### Define the functions to calculate similarity between attributes

In [6]:
def jaccard_list(a, b):
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def name_tokens_similarity(r1, r2):
    s1 = r1.name_string
    s2 = r2.name_string
    if rltk.levenshtein_distance(s1, s2) > min(len(s1), len(s2)) / 3:
            return 0
    return 1

def h_city_string_similarity(r1, r2):
    s1 = r1.h_city_string
    s2 = r2.h_city_string
    if s1 == s2:
        return 1
    return 0

def h_state_string_similarity(r1, r2):
    s1 = r1.h_state_string
    s2 = r2.h_state_string
    if s1 == s2:
        return 1
    return 0

def size_string_similarity(r1, r2):
    s1 = r1.size_string
    s2 = r2.size_string
    if s1 == s2:
        return 1
    return 0

def industry_string_similarity(r1, r2):
    s1 = r1.industry_string
    s2 = r2.industry_string
    words1 = set(s1)
    words2 = set(s2)
    return jaccard_list(words1, words2)

#### Combine multiple similarity functions into a single weightened scoring function:

In [7]:
# threshold value to determine if we are confident the record match
MY_TRESH = 0.8

# entity linkage scoring function
def rule_based_method(r1, r2):
    score_1 = name_tokens_similarity(r1, r2)
    score_2 = h_city_string_similarity(r1, r2)
    score_3 = size_string_similarity(r1, r2)
    score_4 = industry_string_similarity(r1, r2)
    
    total = 0.78 * score_1  + 0.12 * score_2 + 0.1 * score_3
    
    # return two values: boolean if they match or not, float to determine confidence
    return total >= MY_TRESH, total

#### Matching the values using rules-based-method

In [8]:
pairs = []
for r_crunch, r_glass in rltk.get_record_pairs(ds1, ds2):
    result, confidence = rule_based_method(r_crunch, r_glass)
    # print(r_crunch.name_string, r_glass.name_string, result, confidence)
    if result == True:
        pairs.append((r_crunch.id, r_glass.id, r_crunch.name_string, r_glass.name_string, confidence))

In [9]:
len(pairs)

452

### Save the prediction results
#### Save the valid pairs (=matching pairs) to 'Entity_resolution.csv'

In [10]:
header = ['crunchbase.ID', 'glassdoor.ID', 'Crunchbase.CName', 'Glassdoor.CName', 'confidence']
with open(dir_ + 'Entity_resolution.csv', mode='w') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"',quoting=csv.QUOTE_MINIMAL)
    writer.writerow(i for i in header)
    for row in pairs:
        writer.writerow(row)

<hr>

# Tasking2 : Using RDFLib for Knowledge Representation

RDFLib is a Python library for working with RDF, a simple yet powerful language for representing information as graphs. RDFLib aims to be a pythonic RDF API, a Graph is a python collection of RDF Subject, Predicate,  Object Triples.

This notebook introduces simple examples. You can also find additional information in the [official documenation](https://rdflib.readthedocs.io/en/stable/).

### Merge two datasets using the matching pairs

In [11]:
matchDF = pd.read_csv(dir_ + 'Entity_resolution.csv')
matchDF.head()

,crunchbase.ID,glassdoor.ID,Crunchbase.CName,Glassdoor.CName,confidence
0,1,101,3M,3M,1.00
1,2,445,A-Mark Precious Metals,A-Mark Precious Metals,0.88
2,3,259,AECOM,AECOM,1.00
3,5,333,AGCO,AGCO,1.00
4,6,66,AIG,AIG,1.00


In [12]:
crunch = pd.read_csv(crunchbase).drop(columns=['Unnamed: 0.1'])
glass = pd.read_csv(glassdoor).drop(columns=['Unnamed: 0.1'])

#### Get State abbreviation data from wikidata for preprocessing

In [13]:
import requests

url = 'https://query.wikidata.org/sparql'
query = '''
SELECT ?itemLabel ?shortname 
WHERE 
{
  ?item wdt:P31 wd:Q35657 .
  ?item wdt:P1813 ?shortname .
  FILTER (strlen(?shortname) < 3) .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
'''
r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()

state = []
abb = []
for x in data["results"]["bindings"]:
    state.append(x['itemLabel']['value'])
    abb.append(x['shortname']['value'])

stateDF = pd.DataFrame(state, columns=['state'])
stateDF['abbr'] = abb
stateDF

,state,abbr
0,California,CA
1,Alabama,AL
2,Maine,ME
3,New Hampshire,NH
4,Massachusetts,MA
5,Connecticut,CT
6,Hawaii,HI
7,Alaska,AK
8,Florida,FL
9,Arizona,AZ


In [14]:
glass['headquarters_city'] = ''
glass['headquarters_state'] = ''

for idx, area in enumerate(glass['headquarters']):
    try: 
        city = area.split(', ')[0]
        state = area.split(', ')[1]
        
        abbIndex = stateDF[stateDF.abbr.str.contains(state)].index[0]
        glass.loc[idx, 'headquarters_city'] = city
        glass.loc[idx, 'headquarters_state'] = stateDF.loc[abbIndex, 'state']
    except:
        glass.loc[idx, 'headquarters_city'] = city
        glass.loc[idx, 'headquarters_state'] = state

### Construct a knowledge graph, using data matching pairs 
#### Merge 

In [15]:
tot = []
columnTot = list(matchDF.columns[:-1])
columnTot.extend(list(crunch.columns[1:]))
columnTot.extend(list(glass.columns[:]))

for idx in range(len(matchDF)):
    newRow = []
    crunchIdx = list(matchDF.loc[idx,:])[0]
    glassIdx = list(matchDF.loc[idx,:])[1]
    newRow.extend(list(matchDF.iloc[idx,:-1]))
    newRow.extend(list(crunch.iloc[crunchIdx,1:]))
    newRow.extend(list(glass.iloc[glassIdx,:]))
    tot.append(newRow)
    
dfMerge = pd.DataFrame(tot, columns =columnTot)

In [16]:
mustGlass = ['founded','type','revenue']
rmListGlass = [col for col in list(glass.columns) if col not in mustGlass]
rmList = ['crunchbase.ID', 'glassdoor.ID', 'Crunchbase.CName', 'Glassdoor.CName', 'company']
rmList.extend(rmListGlass)

dfMerge = dfMerge.drop(rmList, axis=1)
dfMerge

,name,headquaters_location_city,headquaters_location_state,headquaters_location_country,headquaters_regions,founder,website_link,hub,num_employees_enum,last_funding_type,...,founded_date,operating_status,also_known_as,company_type,legal_name,contact_phone,number_of_exits,founded,type,revenue
0,3M,Saint Paul,Minnesota,United States,"['Greater Minneapolis-Saint Paul Area', 'Great...",[],https://www.3m.com,['3M Alumni Founded Companies'],10000+ Employees,NaN,...,1902,Active,3M Co,For Profit,1902.0,Active,2.0,1902.0,Company - Public (MMM),$10+ billion (USD)
1,A-Mark Precious Metals,Santa Monica,California,United States,"['Greater Los Angeles Area', 'West Coast', 'We...",['Steven C. Markoff'],http://www.amark.com,[],51 to 200 Employees,NaN,...,1965,NaN,Active,For Profit,NaN,NaN,NaN,NaN,Company - Public (AMRK),$5 to $10 billion (USD)
2,AECOM,Los Angeles,California,United States,"['Greater Los Angeles Area', 'West Coast', 'We...",['Richard G. Newman'],http://www.aecom.com/,['AECOM Alumni Founded Companies'],10000+ Employees,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990.0,Company - Public (ACM),$10+ billion (USD)
3,AGCO,Duluth,Georgia,United States,"['Greater Atlanta Area', 'East Coast', 'Southe...",['Robert J. Ratliff'],http://agcocorp.com,[],10000+ Employees,NaN,...,1990,NaN,Active,For Profit,NaN,NaN,NaN,1990.0,Company - Public (AGCO),$10+ billion (USD)
4,AIG,New York,New York,United States,"['Greater New York Area', 'East Coast', 'North...",['Cornelius Vander Starr'],https://www.aig.com,['American International Group Alumni Founded ...,10000+ Employees,NaN,...,"Dec 19, 1919",Active,AIG,For Profit,NaN,NaN,1.0,1919.0,Company - Public (AIG),$10+ billion (USD)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,Yum Brands,Louisville,Kentucky,United States,['Southern US'],['Andrall E. Pearson'],http://www.yum.com,[],10000+ Employees,NaN,...,1997,Active,Tricon Global Restaurants,For Profit,NaN,NaN,1.0,1997.0,Company - Public (YUM),$10+ billion (USD)
448,Yum China Holdings,Shanghai,Shanghai,China,['Asia-Pacific (APAC)'],['Joey Wat'],http://www.yumchina.com/,[],10000+ Employees,Private Equity,...,"May 30, 1997",Active,"百胜中国, Baisheng Zhongguo, 环胜电子商务（上海）有限公司",For Profit,NaN,NaN,1.0,NaN,Company - Public,Unknown / Non-Applicable
449,Zimmer Biomet Holdings,Warsaw,Indiana,United States,"['Great Lakes', 'Midwestern US']","['Bryan Huffman', 'J. J. Ettinger', 'Justin O....",https://zimmer.com,[],10000+ Employees,Post-IPO Debt,...,1927,NaN,Active,For Profit,NaN,NaN,1.0,1927.0,Company - Public (ZBH),$5 to $10 billion (USD)
450,Zoetis,Parsippany,New Jersey,United States,"['Greater New York Area', 'East Coast', 'North...",[],https://www.zoetis.com,[],5001 to 10000 Employees,NaN,...,2014,Active,Pfizer Animal Health,For Profit,NaN,NaN,NaN,2013.0,Company - Public (ZTS),$5 to $10 billion (USD)


In [17]:
dfMerge['headquaters_regions'] = dfMerge['headquaters_regions'].apply(eval).apply(', '.join)
dfMerge['founder'] = dfMerge['founder'].apply(eval).apply(', '.join)
dfMerge['hub'] = dfMerge['hub'].apply(eval).apply(', '.join)
dfMerge['industries'] = dfMerge['industries'].apply(eval).apply(', '.join)
dfMerge

,name,headquaters_location_city,headquaters_location_state,headquaters_location_country,headquaters_regions,founder,website_link,hub,num_employees_enum,last_funding_type,...,founded_date,operating_status,also_known_as,company_type,legal_name,contact_phone,number_of_exits,founded,type,revenue
0,3M,Saint Paul,Minnesota,United States,"Greater Minneapolis-Saint Paul Area, Great Lak...",,https://www.3m.com,3M Alumni Founded Companies,10000+ Employees,NaN,...,1902,Active,3M Co,For Profit,1902.0,Active,2.0,1902.0,Company - Public (MMM),$10+ billion (USD)
1,A-Mark Precious Metals,Santa Monica,California,United States,"Greater Los Angeles Area, West Coast, Western US",Steven C. Markoff,http://www.amark.com,,51 to 200 Employees,NaN,...,1965,NaN,Active,For Profit,NaN,NaN,NaN,NaN,Company - Public (AMRK),$5 to $10 billion (USD)
2,AECOM,Los Angeles,California,United States,"Greater Los Angeles Area, West Coast, Western US",Richard G. Newman,http://www.aecom.com/,AECOM Alumni Founded Companies,10000+ Employees,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990.0,Company - Public (ACM),$10+ billion (USD)
3,AGCO,Duluth,Georgia,United States,"Greater Atlanta Area, East Coast, Southern US",Robert J. Ratliff,http://agcocorp.com,,10000+ Employees,NaN,...,1990,NaN,Active,For Profit,NaN,NaN,NaN,1990.0,Company - Public (AGCO),$10+ billion (USD)
4,AIG,New York,New York,United States,"Greater New York Area, East Coast, Northeaster...",Cornelius Vander Starr,https://www.aig.com,American International Group Alumni Founded Co...,10000+ Employees,NaN,...,"Dec 19, 1919",Active,AIG,For Profit,NaN,NaN,1.0,1919.0,Company - Public (AIG),$10+ billion (USD)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,Yum Brands,Louisville,Kentucky,United States,Southern US,Andrall E. Pearson,http://www.yum.com,,10000+ Employees,NaN,...,1997,Active,Tricon Global Restaurants,For Profit,NaN,NaN,1.0,1997.0,Company - Public (YUM),$10+ billion (USD)
448,Yum China Holdings,Shanghai,Shanghai,China,Asia-Pacific (APAC),Joey Wat,http://www.yumchina.com/,,10000+ Employees,Private Equity,...,"May 30, 1997",Active,"百胜中国, Baisheng Zhongguo, 环胜电子商务（上海）有限公司",For Profit,NaN,NaN,1.0,NaN,Company - Public,Unknown / Non-Applicable
449,Zimmer Biomet Holdings,Warsaw,Indiana,United States,"Great Lakes, Midwestern US","Bryan Huffman, J. J. Ettinger, Justin O. Zimmer",https://zimmer.com,,10000+ Employees,Post-IPO Debt,...,1927,NaN,Active,For Profit,NaN,NaN,1.0,1927.0,Company - Public (ZBH),$5 to $10 billion (USD)
450,Zoetis,Parsippany,New Jersey,United States,"Greater New York Area, East Coast, Northeaster...",,https://www.zoetis.com,,5001 to 10000 Employees,NaN,...,2014,Active,Pfizer Animal Health,For Profit,NaN,NaN,NaN,2013.0,Company - Public (ZTS),$5 to $10 billion (USD)


### Create a graph

In [18]:
import rdflib
MYNS = Namespace('http://dsci558.org/myprojectnamespace#')
RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
SCHEMA = Namespace("https://schema.org/")
WIKIDATA = Namespace("https://wikidata.org/")

In [19]:
my_kg = Graph()
my_kg.bind('shema', SCHEMA) # Schema
my_kg.bind('myns', MYNS)
my_kg.bind('rdfs', RDFS)
my_kg.bind('wiki', WIKIDATA)

### Triple for Company information

#### Define a URI, then add a simple triple to the graph

In [20]:
## Company Entities
companyList = []
companyList.extend(glass['company'])
companyList.extend(crunch['name'])
companyList = list(set(companyList))
                            
for company in companyList:
    company_rename = re.sub("[^1-9a-zA-Z&]", "_", company)
    company_uri = URIRef(MYNS[company_rename])
    my_kg.add((company_uri, RDF.type, MYNS['company']))
    my_kg.add((company_uri, RDF.type, SCHEMA.Corporation))
    my_kg.add((company_uri, RDFS.label, Literal(company)))

    
## State Entities
for state in stateDF['state']:
    state = re.sub("[^1-9a-zA-Z&]", "_", state)
    state_uri = URIRef(MYNS[state])
    my_kg.add((state_uri, RDF.type, MYNS['Headquarter_state']))
    my_kg.add((state_uri, RDF.type, SCHEMA.State))
    my_kg.add((state_uri, RDFS.label, Literal(state)))
    
              
## City Entities
cityList = []
cityList.extend(glass['headquarters_city'].fillna("nan"))
cityList.extend(crunch['headquaters_location_city'].fillna("nan"))
cityList = list(set(cityList))
for city in cityList[1:]:
    city = re.sub("[^1-9a-zA-Z&]", "_", city)
    city_uri = URIRef(MYNS[city])
    my_kg.add((city_uri, RDF.type, MYNS['Headquarter_city']))
    my_kg.add((city_uri, RDF.type, SCHEMA.City))
    my_kg.add((city_uri, RDFS.label, Literal(city)))
    

## Country Entities
countryList = []
countryList.extend(crunch['headquaters_location_country'].fillna("nan"))
countryList = list(set(countryList))
for country in countryList:
    country = re.sub("[^1-9a-zA-Z&]", "_", country)
    country_uri = URIRef(MYNS[country])
    my_kg.add((country_uri, RDF.type, MYNS['Headquarter_city']))
    my_kg.add((country_uri, RDF.type, SCHEMA.Country))
    my_kg.add((country_uri, RDFS.label, Literal(country)))


## Add other predicates
predicateList = list(dfMerge.columns)
removeList = ['headquaters_location_city', 'headquaters_location_state', 'headquaters_location_country', 'headquaters_regions']
for rm in removeList:
    predicateList.remove(rm)
for predicate in predicateList:
    predicate_rename = re.sub("[^1-9a-zA-Z&]", "_", predicate)
    predicate_uri = MYNS[predicate_rename]
    my_kg.add((predicate_uri, RDF.type, RDF.Property))
    my_kg.add((predicate_uri, RDFS.label, Literal(predicate, lang = 'en')))

In [21]:
for idx, row in dfMerge.iterrows():
    company_uri = re.sub("[^1-9a-zA-Z&]", "_", row['name'])
    
    colList = list(dfMerge.columns)[1:]
    for col in colList:
        obj = row[col]
        if str(obj) != 'nan' and str(obj) != '[]':
            try:
                obj = str(int(obj))            
                my_kg.add((MYNS[company_uri], MYNS[col], Literal(obj)))  
            except:
                obj = str(obj)          
                my_kg.add((MYNS[company_uri], MYNS[col], Literal(obj)))  

#### Dump our graph triples into some `ttl` file:

In [22]:
my_kg.serialize('company.ttl', format="turtle") 

<Graph identifier=N1086ff8801f945a69b76e989b03a837b (<class 'rdflib.graph.Graph'>)>

### Triple for Job postings

#### Define a URI, then add a simple triple to the graph

In [23]:
def get_token(string):
    data = string.split()
    return data[0]

def get_state(string):
    data = string.strip().split(',')
    if len(data) > 1:
        state = data[-1].strip()
        try:
            index = list(stateDF['abbr']).index(state)
            state = stateDF.loc[index,'state']
            return state
        except: 
            return state
        return 
    else:
        return 'Remote'
    
def get_city(string):
    data = string.strip().split(',')
    if len(data) > 1:
        city = data[0].strip()
        return city
    else:
        return None

In [24]:
job = pd.read_csv('./data/job_info.csv').drop(columns=['Unnamed: 0'])
job['State'] = job['Location'].apply(lambda x: get_state(x))
job['City'] = job['Location'].apply(lambda x: get_city(x))
job['Company_token'] = job['Company'].apply(lambda x: get_token(x))
job['Company_final'] = job['Company'].copy()

job.tail()

,Title,Company,Company_Link,Location,Company_Score,Salary_LowerBound,Salary_HigherBound,Salary_Type,Post_Duration,Job_Type,Application_Link,Job_Description,Programming_Languages,Soft_Requirements,Framework_Requirements,State,City,Company_token,Company_final
29358,"Sr. Analyst, Learning Analytics",Comcast,http://www.comcastcareers.com/,"Englewood, CO",3.9,78.0,107.0,Year,31,Information Security Analyst,www.glassdoor.com/partner/jobListing.htm?pos=1...,"Comcast’s Technology, Product & Experience org...",['Python'],['database'],['Spark'],Colorado,Englewood,Comcast,Comcast
29359,"Sr. Analyst, Learning Analytics",Comcast,http://www.comcastcareers.com/,"Englewood, CO",3.9,78.0,107.0,Year,8,Information Security Analyst,www.glassdoor.com/partner/jobListing.htm?pos=1...,Comcast Business offers technology solutions r...,['SQL'],['database'],['Hadoop'],Colorado,Englewood,Comcast,Comcast
29360,"Sr. Analyst, Learning Analytics",Effectv,http://www.effectv.com/,"Englewood, CO",3.7,78.0,107.0,Year,14,Information Security Analyst,www.glassdoor.com/partner/jobListing.htm?pos=1...,"Effectv, the advertising sales division of Com...","['Python', 'SQL']","['work experience', 'bachelor']",['Linux'],Colorado,Englewood,Effectv,Effectv
29361,"Sr. Analyst, Learning Analytics",Comcast,http://www.comcastcareers.com/,"Englewood, CO",3.9,78.0,107.0,Year,31,Information Security Analyst,www.glassdoor.com/partner/jobListing.htm?pos=1...,Comcast brings together the best in media and ...,"['Python', 'SQL']",['database'],['Spark'],Colorado,Englewood,Comcast,Comcast
29362,"Sr. Analyst, Learning Analytics",Comcast,http://www.comcastcareers.com/,"Englewood, CO",3.9,78.0,107.0,Year,18,Information Security Analyst,www.glassdoor.com/partner/jobListing.htm?pos=1...,Comcast brings together the best in media and ...,"['Python', 'SQL']","['work experience', 'bachelor']",['Spark'],Colorado,Englewood,Comcast,Comcast


In [25]:
len(list(np.setdiff1d(np.array(job['Company']), np.array(dfMerge['name']))))

2709

#### Preprocessing to matching the compaines

In [26]:
dfMerge['website_link_prepro'] = dfMerge['website_link'].str.replace('https:','')
dfMerge['website_link_prepro'] = dfMerge['website_link_prepro'].str.replace('http:','')
dfMerge['website_link_prepro'] = dfMerge['website_link_prepro'].str.replace('/','')
dfMerge['website_link_prepro'] = dfMerge['website_link_prepro'].str.replace('www.','')

job['Company_Link_prepro'] = job['Company_Link'].str.replace('https:','')
job['Company_Link_prepro'] = job['Company_Link_prepro'].str.replace('http:','')
job['Company_Link_prepro'] = job['Company_Link_prepro'].str.replace('/','')
job['Company_Link_prepro'] = job['Company_Link_prepro'].str.replace('www.','')

In [27]:
company_matching = {}
for idx, row in dfMerge.iterrows():
    company_matching[row['website_link_prepro']] = row['name']
    
from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [28]:
NoNeedCompany = []
CompanyDB = list(dfMerge['name'])

for idx, row in job.iterrows():
    if row['Company'] not in CompanyDB:
        if row['Company_Link_prepro'] in list(company_matching.keys()):
            job.at[idx, 'Company_final'] = company_matching[row['Company_Link_prepro']]
            continue
            
        for website, cname in company_matching.items():
            try:
                score = similar(row['Company_Link_prepro'], website)
                if score >= 0.7:
                    job.at[idx, 'Company_final'] = cname
                    break
            except:
                NoNeedCompany.append(idx)

#### Drop the companies which do not have any info

In [29]:
job = job.drop(NoNeedCompany).reset_index(drop=True)
job.head()

,Title,Company,Company_Link,Location,Company_Score,Salary_LowerBound,Salary_HigherBound,Salary_Type,Post_Duration,Job_Type,Application_Link,Job_Description,Programming_Languages,Soft_Requirements,Framework_Requirements,State,City,Company_token,Company_final,Company_Link_prepro
0,Software Engineer II,Honeywell,http://www.honeywell.com/,"Aguadilla, PR",4.0,61.0,81.0,Year,3,software developer,www.glassdoor.com/partner/jobListing.htm?pos=1...,"Join a team recognized for leadership, innovat...","['Java', 'JavaScript', 'SQL', 'CSS']","['work experience', 'bachelor', 'database']",['Redis'],PR,Aguadilla,Honeywell,Honeywell International,honeywell.com
1,Software Engineer II,Honeywell,http://www.honeywell.com/,"Aguadilla, PR",4.0,61.0,81.0,Year,28,software developer,www.glassdoor.com/partner/jobListing.htm?pos=1...,Advanced Cloud Developer\nAdvanced Cloud Devel...,['Java'],"['work experience', 'bachelor']",['JUnit'],PR,Aguadilla,Honeywell,Honeywell International,honeywell.com
2,Software Engineer II,The RMR Group,http://www.rmrgroup.com/,"Aguadilla, PR",3.7,61.0,81.0,Year,10,software developer,www.glassdoor.com/partner/jobListing.htm?pos=1...,Overview:\nSeeking a highly motivated building...,['Python'],"['bachelor', 'database']",['Spark'],PR,Aguadilla,The,Citigroup,rmrgroup.com
3,Building Automation Application Engineer (Loca...,The RMR Group,http://www.rmrgroup.com/,"Newton, MA",3.7,54.0,73.0,Year,10,Cloud Computing Engineer,www.glassdoor.com/partner/jobListing.htm?pos=1...,Overview:\nSeeking a highly motivated building...,['Python'],"['bachelor', 'database']",['Spark'],Massachusetts,Newton,The,Citigroup,rmrgroup.com
4,Building Automation Application Engineer (Loca...,Honeywell,http://www.honeywell.com/,"Newton, MA",4.0,54.0,73.0,Year,21,Cloud Computing Engineer,www.glassdoor.com/partner/jobListing.htm?pos=1...,"Join a team recognized for leadership, innovat...","['Java', 'HTML']",['front end'],['Spark'],Massachusetts,Newton,Honeywell,Honeywell International,honeywell.com


In [30]:
list1 = map(lambda x: x.capitalize(), job['Job_Type'])
job['Job_Type'] = list(list1)

In [31]:
job['Programming_Languages'] = job['Programming_Languages'].apply(eval).apply(', '.join)
job['Soft_Requirements'] = job['Soft_Requirements'].apply(eval).apply(', '.join)
job['Framework_Requirements'] = job['Framework_Requirements'].apply(eval).apply(', '.join)
job

,Title,Company,Company_Link,Location,Company_Score,Salary_LowerBound,Salary_HigherBound,Salary_Type,Post_Duration,Job_Type,Application_Link,Job_Description,Programming_Languages,Soft_Requirements,Framework_Requirements,State,City,Company_token,Company_final,Company_Link_prepro
0,Software Engineer II,Honeywell,http://www.honeywell.com/,"Aguadilla, PR",4.0,61.0,81.0,Year,3,Software developer,www.glassdoor.com/partner/jobListing.htm?pos=1...,"Join a team recognized for leadership, innovat...","Java, JavaScript, SQL, CSS","work experience, bachelor, database",Redis,PR,Aguadilla,Honeywell,Honeywell International,honeywell.com
1,Software Engineer II,Honeywell,http://www.honeywell.com/,"Aguadilla, PR",4.0,61.0,81.0,Year,28,Software developer,www.glassdoor.com/partner/jobListing.htm?pos=1...,Advanced Cloud Developer\nAdvanced Cloud Devel...,Java,"work experience, bachelor",JUnit,PR,Aguadilla,Honeywell,Honeywell International,honeywell.com
2,Software Engineer II,The RMR Group,http://www.rmrgroup.com/,"Aguadilla, PR",3.7,61.0,81.0,Year,10,Software developer,www.glassdoor.com/partner/jobListing.htm?pos=1...,Overview:\nSeeking a highly motivated building...,Python,"bachelor, database",Spark,PR,Aguadilla,The,Citigroup,rmrgroup.com
3,Building Automation Application Engineer (Loca...,The RMR Group,http://www.rmrgroup.com/,"Newton, MA",3.7,54.0,73.0,Year,10,Cloud computing engineer,www.glassdoor.com/partner/jobListing.htm?pos=1...,Overview:\nSeeking a highly motivated building...,Python,"bachelor, database",Spark,Massachusetts,Newton,The,Citigroup,rmrgroup.com
4,Building Automation Application Engineer (Loca...,Honeywell,http://www.honeywell.com/,"Newton, MA",4.0,54.0,73.0,Year,21,Cloud computing engineer,www.glassdoor.com/partner/jobListing.htm?pos=1...,"Join a team recognized for leadership, innovat...","Java, HTML",front end,Spark,Massachusetts,Newton,Honeywell,Honeywell International,honeywell.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20688,"Sr. Analyst, Learning Analytics",Comcast,http://www.comcastcareers.com/,"Englewood, CO",3.9,78.0,107.0,Year,1,Information security analyst,www.glassdoor.com/partner/jobListing.htm?pos=1...,Comcast brings together the best in media and ...,SQL,database,Spark,Colorado,Englewood,Comcast,Comcast,comcastcareers.com
20689,"Sr. Analyst, Learning Analytics",Comcast,http://www.comcastcareers.com/,"Englewood, CO",3.9,78.0,107.0,Year,31,Information security analyst,www.glassdoor.com/partner/jobListing.htm?pos=1...,"Comcast’s Technology, Product & Experience org...",Python,database,Spark,Colorado,Englewood,Comcast,Comcast,comcastcareers.com
20690,"Sr. Analyst, Learning Analytics",Comcast,http://www.comcastcareers.com/,"Englewood, CO",3.9,78.0,107.0,Year,8,Information security analyst,www.glassdoor.com/partner/jobListing.htm?pos=1...,Comcast Business offers technology solutions r...,SQL,database,Hadoop,Colorado,Englewood,Comcast,Comcast,comcastcareers.com
20691,"Sr. Analyst, Learning Analytics",Comcast,http://www.comcastcareers.com/,"Englewood, CO",3.9,78.0,107.0,Year,31,Information security analyst,www.glassdoor.com/partner/jobListing.htm?pos=1...,Comcast brings together the best in media and ...,"Python, SQL",database,Spark,Colorado,Englewood,Comcast,Comcast,comcastcareers.com


In [32]:
# ## Job posting Entities
# for idx, jobposting in enumerate(job['Title']):
#     jobname = 'job_' + str(idx)
#     job_uri = URIRef(MYNS[jobname])
#     my_kg.add((job_uri, RDF.type, MYNS['jobs']))
#     my_kg.add((job_uri, RDFS.label, Literal(jobname)))
    
## Add other predicates
predicateList = list(job.columns)
removeList = ['Company','Location', 'State', 'City', 'Company_token', 'Company_final', 'Company_Link', 'Company_Link_prepro']
for rm in removeList:
    predicateList.remove(rm)
for predicate in predicateList:
    predicate_rename = re.sub("[^1-9a-zA-Z&.]", "_", predicate)
    predicate_uri = MYNS[predicate_rename]
    my_kg.add((predicate_uri, RDF.type, RDF.Property))
    my_kg.add((predicate_uri, RDFS.label, Literal(predicate, lang = 'en')))

    if predicate == 'State':
        my_kg.add((predicate_uri, RDF.type, SCHEMA.State))
    if predicate == 'City':
        my_kg.add((predicate_uri, RDF.type, SCHEMA.City))

In [33]:
for idx, row in job.iterrows():
    jobname = 'job_' + str(idx)
    job_uri = URIRef(MYNS[jobname])
    my_kg.add((job_uri, RDF.type, MYNS['jobs']))
    my_kg.add((job_uri, RDFS.label, Literal(jobname)))
    
    colList = list(job.columns)
    for col in colList:
        if col in ['Company','Location', 'Company_token', 'Company_Link', 'Company_Link_prepro']:
            continue
        
        obj = row[col]
        if col == 'Company_final':
            col_rename = 'Company'
            obj_rename = re.sub("[^1-9a-zA-Z&]", "_", obj)
            my_kg.add((MYNS[job_uri], MYNS['Company'], MYNS[obj_rename]))
            my_kg.add((MYNS[job_uri], MYNS['Company'], Literal(obj)))
            
        else:
            if str(obj) != 'nan' and str(obj) != '[]':
                try:
                    obj = str(int(obj))            
                    my_kg.add((MYNS[job_uri], MYNS[col], Literal(obj)))  
                except:
                    obj = str(obj)
                    my_kg.add((MYNS[job_uri], MYNS[col], Literal(obj)))  

In [34]:
my_kg.serialize('KG2.ttl', format="turtle") 

<Graph identifier=N1086ff8801f945a69b76e989b03a837b (<class 'rdflib.graph.Graph'>)>